In [1]:
import os
import datetime
import json
from typing import Literal, TypedDict, Annotated, Dict

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain_core.language_models import BaseChatModel
from langchain_core.messages import ToolMessage, HumanMessage
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.output_parsers import BaseOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import StructuredTool
from langgraph.prebuilt import ToolNode

from langgraph.graph import END, StateGraph, START
from langgraph.graph.message import add_messages

from pydantic import BaseModel, Field, ValidationError

In [2]:
load_dotenv()

True

In [3]:
# Định nghĩa LLM sử dụng
llm = ChatOpenAI(model="gpt-4o", api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
# Định nghĩa công cụ tìm kiếm
search = DuckDuckGoSearchAPIWrapper()
search_tool = DuckDuckGoSearchResults(api_wrapper=search, max_results=5)

In [5]:
class Reflection(BaseModel):
    missing: str = Field(
		..., 
		title="Thiếu sót", 
		description="Những thông tin quan trọng còn thiếu trong câu trả lời."
	)
    superfluous: str = Field(
		..., 
		title="Dư thừa", 
		description="Những thông tin không cần thiết hoặc quá dài dòng trong câu trả lời."
	)
    unclear: str = Field(
		..., 
		title="Mơ hồ", 
		description="Những phần nội dung chưa rõ ràng hoặc gây khó hiểu trong câu trả lời."
	)

class Response(BaseModel):    
    answer: str = Field(
        ...,
        title="Câu trả lời",
        description="Câu trả lời chi tiết cho câu hỏi."
    )
    reflection: Reflection = Field(
        ...,
        title="Phản ánh",
        description="Đánh giá về câu trả lời ban đầu, bao gồm những điểm thiếu sót, dư thừa và chưa rõ ràng."
    )
    search_queries: list[str] = Field(
        ...,
        title="Truy vấn tìm kiếm",
        description="1-3 truy vấn tìm kiếm (search query) để nghiên cứu và cải thiện câu trả lời dựa trên đánh giá phản ánh."
    )

class Revise(Response):
    """
	- Chỉnh sửa câu trả lời ban đầu của bạn cho câu hỏi. 
	- Cung cấp một câu trả lời, phản ánh, trích dẫn phản ánh của bạn với các tài liệu tham khảo.
	- Cuối cùng, thêm các truy vấn tìm kiếm để cải thiện câu trả lời.
	"""
    references: list[str] = Field(
		...,
		title="Tài liệu tham khảo",
        description="Các trích dẫn hỗ trợ cho câu trả lời đã cập nhật của bạn."
    )

In [6]:
class Responder:
	def __init__(self, llm: BaseChatModel, parser: BaseOutputParser):
		self.llm = llm
		self.parser = parser

	def respond(self, state: Dict) -> Dict:
		response = []
		for attempt in range(3):
			response = self.llm.invoke(
				{"messages": state["messages"]},
				{"tags": [f"attempt:{attempt}"]}
			)

			if hasattr(response, "tool_calls") and response.tool_calls:
				if len(response.tool_calls) > 1:
					response.tool_calls = [response.tool_calls[0]]

			try:
				self.parser.invoke(response)
				return {"messages": response}
			except ValidationError as e:
				state["messages"] = state["messages"] + [
					response,
					ToolMessage(
						content=f"{repr(e)}\n\nHãy chú ý đến cấu trúc hàm.\n\n"
						+ json.dumps(self.parser.model_json_schema())
						+ "\nHãy phản hồi bằng cách sửa tất cả các lỗi hợp lệ (validation errors).",
						tool_call_id=response.tool_calls[0]["id"],
					),
				]
		return {"messages": response}

In [7]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
			<instruction>
			Bạn là một chuyên gia nghiên cứu.
			Thời gian hiện tại: {time}

			1. {first_instruction}
			2. Phản ánh và phê bình câu trả lời của bạn. Hãy nghiêm khắc để tối đa hóa sự cải thiện.
			3. Đề xuất các truy vấn tìm kiếm để nghiên cứu thông tin và cải thiện câu trả lời của bạn.
			</instruction>
			""",
        ),
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            """
			<reminder>
			- Phản ánh về câu hỏi ban đầu của người dùng và các hành động đã thực hiện cho đến nay. 
			- Phản hồi bằng cách sử dụng hàm {function_name}.
			</reminder> 
			""",
        ),
    ]
).partial(
	time=lambda: datetime.datetime.now().isoformat(),
)

initial_response = prompt_template.partial(
    first_instruction="Cung cấp một câu trả lời chi tiết.",
    function_name=Response.__name__,
) | llm.bind_tools(tools=[Response])

response_parser = PydanticToolsParser(tools=[Response])

first_responder = Responder(
    llm=initial_response,
	parser=response_parser,
)

In [8]:
revise_instructions = (
	"""
	<instruction>
	- Bản PHẢI chỉnh sửa câu trả lời trước đó của bạn bằng cách bổ sung thêm thông tin mới.
    - Bạn cần sử dụng phê bình trước đó để thêm thông tin quan trọng vào câu trả lời của mình.
    - BẠN PHẢI bao gồm các trích dẫn đánh số thứ tự trong câu trả lời đã chỉnh sửa để đảm bảo nó có thể được xác minh.
    - Thêm một phần "Tài liệu tham khảo" ở cuối câu trả lời của bạn. Dưới dạng:
        - [1] https://example.com
        - [2] https://example.com
	- Phải đảm bảo rằng câu trả lời của bạn hoàn chỉnh và chi tiết.
	</instruction>
	"""
)

revision = prompt_template.partial(
    first_instruction=revise_instructions,
    function_name=Revise.__name__,
) | llm.bind_tools(tools=[Revise])

revision_parser = PydanticToolsParser(tools=[Revise])

revisor = Responder(
	llm=revision,
	parser=revision_parser,
)

In [9]:
example_question = "Tại sao bầu trời màu xanh?"

initial = first_responder.respond(
    {"messages": [HumanMessage(content=example_question)]}
)

revised = revisor.respond(
    {
        "messages": [
            HumanMessage(content=example_question),
            initial["messages"],
            ToolMessage(
                tool_call_id=initial["messages"].tool_calls[0]["id"],
                content=json.dumps(
                    search_tool.invoke(
                        {
                            "query": initial["messages"].tool_calls[0]["args"][
                                "search_queries"
                            ][0]
                        }
                    )
                ),
            ),
        ]
    }
)

revised["messages"]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2jdvSw2gXAAJuWhsjaL4o3pu', 'function': {'arguments': '{"answer":"Bầu trời có màu xanh chủ yếu là do hiện tượng tán xạ Rayleigh. Khi ánh sáng mặt trời đi qua bầu khí quyển, các phần tử nhỏ như phân tử khí nitơ và oxy tán xạ các bước sóng ngắn của ánh sáng mặt trời, tức là ánh sáng màu xanh và tím, mạnh hơn các bước sóng dài như đỏ, cam. \\n\\nMặc dù cả ánh sáng xanh và tím đều bị tán xạ, mắt người nhạy cảm với ánh sáng xanh hơn tím, và đồng thời, ánh sáng tím phần lớn bị tầng trên của khí quyển hấp thụ, vì vậy bầu trời có màu xanh [1]. \\n\\nHơn nữa, sự tán xạ này khiến cho màu sắc của bầu trời thay đổi vào những thời điểm khác nhau trong ngày. Khi mặt trời ở vị trí thấp hơn trên bầu trời, chẳng hạn như vào lúc bình minh hoặc hoàng hôn, ánh sáng phải đi qua một lớp khí quyển dày hơn, dẫn đến tán xạ nhiều hơn ánh sáng đỏ và vàng, làm cho bầu trời xuất hiện màu sắc ấm hơn như màu đỏ hoặc cam [2].\\n\\nTrong một nghiên cứ

In [10]:
def run_queries(search_queries: list[str], **kwargs):
    """Chạy các truy vấn tìm kiếm và trả về kết quả."""
    return search_tool.batch([{"query": query} for query in search_queries])

tool_node = ToolNode(
    [
        StructuredTool.from_function(run_queries, name=Response.__name__),
        StructuredTool.from_function(run_queries, name=Revise.__name__),
    ]
)

In [11]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

MAX_ITERATIONS = 5

def _get_num_iterations(state: list):
    i = 0
    for m in state[::-1]:
        if m.type not in {"tool", "ai"}:
            break
        i += 1
    return i

def event_loop(state: list):
    num_iterations = _get_num_iterations(state["messages"])
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"

workflow = StateGraph(State)

workflow.add_node("draft", first_responder.respond)
workflow.add_node("execute_tools", tool_node)
workflow.add_node("revise", revisor.respond)

workflow.add_edge(START, "draft")
workflow.add_edge("draft", "execute_tools")
workflow.add_edge("execute_tools", "revise")

workflow.add_conditional_edges("revise", event_loop, ["execute_tools", END])

graph = workflow.compile()

In [12]:
events = graph.stream(
    {"messages": [("user", "Tại sao doanh thu của Viettel trong những năm gần đây lại tăng ở mức tốt?")]},
    stream_mode="values",
)

for i, step in enumerate(events):
    print(f"Bước {i}")
    step["messages"][-1].pretty_print()

Bước 0
================================ Human Message =================================

Tại sao doanh thu của Viettel trong những năm gần đây lại tăng ở mức tốt?
Bước 1
================================== Ai Message ==================================
Tool Calls:
  Response (call_AuxrVNx3CvqZXHJupB9dnEno)
 Call ID: call_AuxrVNx3CvqZXHJupB9dnEno
  Args:
    answer: Doanh thu của Viettel trong những năm gần đây tăng trưởng tốt chủ yếu nhờ vào một số yếu tố sau:

1. **Đầu tư vào công nghệ mới:** Viettel luôn đầu tư mạnh mẽ vào công nghệ hiện đại và tiên tiến, chẳng hạn như 4G, 5G, và các công nghệ IoT. Điều này giúp thu hút thêm nhiều khách hàng và mở rộng các dịch vụ cung cấp, từ đó tăng doanh thu.

2. **Mở rộng thị trường quốc tế:** Viettel đã mở rộng hoạt động của mình sang nhiều thị trường quốc tế, đặc biệt là ở khu vực Châu Phi và Châu Á, nơi nhu cầu về dịch vụ viễn thông vẫn đang tăng nhanh. Các hoạt động quốc tế này đã đóng góp một phần quan trọng vào tăng trưởng doanh thu của Viett